In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# for calculating Polarity and Subjectivity
from textblob import TextBlob
import seaborn as sns
import wordcloud
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [8]:
df=pd.read_csv('dataset.csv')

In [9]:
print("Shape of Data:",df.shape)
df.head(10)

Shape of Data: (200000, 2)


,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False
5,"Martha stewart tweets hideous food photo, twit...",False
6,What is a pokemon master's favorite kind of pa...,True
7,Why do native americans hate it when it rains ...,True
8,"Obama's climate change legacy is impressive, i...",False
9,"My family tree is a cactus, we're all pricks.",True


In [10]:
def get_polarity(text):
    textblob = TextBlob(str(text.encode('utf-8')))
    pol = textblob.sentiment.polarity
    return pol

# lets apply the function
df['polarity'] = df['text'].apply(get_polarity)

In [11]:
def get_subjectivity(text):
    textblob = TextBlob(str(text.encode('utf-8')))
    subj = textblob.sentiment.subjectivity
    return subj

# lets apply the Function
df['subjectivity'] = df['text'].apply(get_subjectivity)

In [12]:
df[['polarity','subjectivity']].describe()

,polarity,subjectivity
count,200000.000000,200000.000000
mean,0.052513,0.309548
std,0.288698,0.330089
min,-1.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.233333
75%,0.136364,0.538889
max,1.000000,1.000000


In [13]:
def remove_special_characters(text):
    pat = r'[^a-zA-z0-9]' 
    return re.sub(pat, ' ', text)


df['text'] = df.apply(lambda x: remove_special_characters(x['text']), axis = 1)

In [14]:
import re
def drop_numbers(list_text):
    list_text_new = []
    for i in list_text:
        if not re.search('\d', i):
            list_text_new.append(i)
    return ''.join(list_text_new)

df['text'] = df['text'].apply(drop_numbers)

In [15]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Balla/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\Balla\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
df["text"] = df["text"].map(remove_stopwords)
df.text

In [16]:
# creating bag of words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 2500)

x = cv.fit_transform(df['text']).toarray()
y = df['humor'].values
print(x.shape)
print(y.shape)

(200000, 2500)
(200000,)


In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
pred=classifier.predict(X_test)
pred

In [ ]:
print('model_DT Train Score is : ' , classifier.score(X_train, y_train))
print('model_DT Test Score is : ' , classifier.score(X_test, y_test))

In [ ]:
from sklearn.metrics import accuracy_score
print("The Score is: ",accuracy_score(y_test , pred)*100)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix #for model evaluation
confusion_matrix=confusion_matrix(y_test,pred)
confusion_matrix

sns.heatmap(confusion_matrix)

In [ ]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier1 = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier1.fit(X_train, y_train)

In [ ]:
y_pred = classifier1.predict(X_test)
y_pred

In [ ]:
print('model_DT Train Score is : ' , classifier1.score(X_train, y_train))
print('model_DT Test Score is : ' , classifier1.score(X_test, y_test))

In [ ]:
print("The Score is: ",accuracy_score(y_test , y_pred)*100)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix #for model evaluation
confusion_matrix=confusion_matrix(y_test,y_pred)
confusion_matrix

In [ ]:
import matplotlib

matplotlib.rcParams['figure.figsize'] = (8,5)
plot_confusion_matrix(classifier1,X_test,y_test);